## End of Module Assessment 3

Your org implemented a Google Form for your Recweek, but it was not able to design it well enough to prevent ambiguous data entry.You are tasked to clean up a csv of fake, dirty data called **recweekdata.csv** (presumably exported from some spreadsheet).

In addition, you have a file named **studentprofiles.csv** which has first and last names plus the genders of the students.

The output of this assignment is to convert recweekdata.csv into usable data for statistics. Submit the following files through GitHub:
1. A cleaned recweekdata.csv called **cleanrecweekdata.csv**
2. This notebook, containing the code (properly commented) that you used to clean the data

Your solution **must use Pandas and possibly Numpy**.

Avoid using loops as much as you can. If I see loops where Pandas vectorization operations are more appropriate, you will receive deductions. 

Hint(s):
* if you need to apply scalar (regular single-value) functions on Pandas columns, use the Pandas Dataframe .apply(...) function

Clearly show your output similiar to what you see in this notebook. 

#### Required Statistics

1. How many people were born in each month?
2. How many people were born on each day?
3. How many people were born in each year?
4. How many people come from each course?
5. How many people per year level?
6. How many people have numbers from Globe (0917 or 0927) and Smart (0918)? *Note: it's up to you how to format this output*
7. How many Globe subscribers are: Male? Female?
8. How many Smart subscribers are: Male? Female?


In [1]:
# start of code; add as many cells as needed
import pandas as pd
import re
import numpy as np

In [1]:
# the outputs below use the dataframe variable name recweek

In [4]:
# check header

,firstname,lastname,birthmonth,birthday,birthyear,course,yearlevel,phonenumber
0,name1,lastname1,11,26,1998,BS LM,2,+63 918 302 6210
1,name2,lastname2,May,26,2001,Management,4,927 212 6276
2,name3,lastname3,5,16,1997,BS CTM,1,+63 918 253 7017
3,name4,lastname4,Dec,9,97,BS ITE,3,918 714 4360
4,name5,lastname5,Sep,5,97,BS CTM,3,+63 927 462 1674


In [2]:
# write code here
filename = "recweekdata.csv"
recweek = pd.read_csv(filename)
recweek.head()

,firstname,lastname,birthmonth,birthday,birthyear,course,yearlevel,phonenumber
0,name1,lastname1,11,26,1998,BS LM,2,+63 918 302 6210
1,name2,lastname2,May,26,2001,Management,4,927 212 6276
2,name3,lastname3,5,16,1997,BS CTM,1,+63 918 253 7017
3,name4,lastname4,Dec,9,97,BS ITE,3,918 714 4360
4,name5,lastname5,Sep,5,97,BS CTM,3,+63 927 462 1674


In [5]:
# check tail

,firstname,lastname,birthmonth,birthday,birthyear,course,yearlevel,phonenumber
2995,name2996,lastname2996,March,28,97,BS CTM,2,917 795 5051
2996,name2997,lastname2997,May,19,1998,LM,2,917 445 3378
2997,name2998,lastname2998,July,3,98,ITE,2,917 154 5694
2998,name2999,lastname2999,12,2,98,ME,3,927 497 8977
2999,name3000,lastname3000,Jul,13,1997,Legal Management,1,+63 918 719 7624


In [3]:
# write code here
recweek.tail()


,firstname,lastname,birthmonth,birthday,birthyear,course,yearlevel,phonenumber
2995,name2996,lastname2996,March,28,97,BS CTM,2,917 795 5051
2996,name2997,lastname2997,May,19,1998,LM,2,917 445 3378
2997,name2998,lastname2998,July,3,98,ITE,2,917 154 5694
2998,name2999,lastname2999,12,2,98,ME,3,927 497 8977
2999,name3000,lastname3000,Jul,13,1997,Legal Management,1,+63 918 719 7624


In [9]:
# Clean Month

# dump recweek data
recweek

,firstname,lastname,birthmonth,birthday,birthyear,course,yearlevel,phonenumber
0,name1,lastname1,November,26,1998,BS LM,2,+63 918 302 6210
1,name2,lastname2,May,26,2001,Management,4,927 212 6276
2,name3,lastname3,May,16,1997,BS CTM,1,+63 918 253 7017
3,name4,lastname4,December,9,97,BS ITE,3,918 714 4360
4,name5,lastname5,September,5,97,BS CTM,3,+63 927 462 1674
...,...,...,...,...,...,...,...,...
2995,name2996,lastname2996,March,28,97,BS CTM,2,917 795 5051
2996,name2997,lastname2997,May,19,1998,LM,2,917 445 3378
2997,name2998,lastname2998,July,3,98,ITE,2,917 154 5694
2998,name2999,lastname2999,December,2,98,ME,3,927 497 8977


In [4]:
# write code here

# The various syntax for most of the months are kept in a consistent format:

# Case 1 - Three character abbreviations  (Jan)
# Case 2 - Number of month designation    (1)
# Case 3 - The whole month spelled out    (January)

# Capitalization and format is kept throughout. Data validation takes into account Case 3 as well so that
# "None" types are not returned.

# Confirming the data types of each column
recweek.dtypes

# A dictionary containing all the possible dirty cases for each month
monthsdict = {
    "Jan":"January","Feb":"February","Mar":"March",
    "Apr":"April","May":"May","Jun":"June",
    "Jul":"July","Aug":"August","Sep":"September",
    "Oct":"October","Nov":"November","Dec":"December",
    "1":"January","2":"February","3":"March",
    "4":"April","5":"May","6":"June",
    "7":"July","8":"August","9":"September",
    "10":"October","11":"November","12":"December",
    "January":"January","February":"February","March":"March",
    "April":"April","May":"May","June":"June",
    "July":"July","August":"August","September":"September",
    "October":"October","November":"November","December":"December"}

# Function that replaces dirty month via vectorization
def cleanmonths(month):
    for m in monthsdict:
        if month==m:
            return str(monthsdict[m])
            continue
        else:
            continue

# Vectorizing months
recweek["birthmonth"] = recweek["birthmonth"].apply(cleanmonths) 

# Dump 
recweek

,firstname,lastname,birthmonth,birthday,birthyear,course,yearlevel,phonenumber
0,name1,lastname1,November,26,1998,BS LM,2,+63 918 302 6210
1,name2,lastname2,May,26,2001,Management,4,927 212 6276
2,name3,lastname3,May,16,1997,BS CTM,1,+63 918 253 7017
3,name4,lastname4,December,9,97,BS ITE,3,918 714 4360
4,name5,lastname5,September,5,97,BS CTM,3,+63 927 462 1674
...,...,...,...,...,...,...,...,...
2995,name2996,lastname2996,March,28,97,BS CTM,2,917 795 5051
2996,name2997,lastname2997,May,19,1998,LM,2,917 445 3378
2997,name2998,lastname2998,July,3,98,ITE,2,917 154 5694
2998,name2999,lastname2999,December,2,98,ME,3,927 497 8977


In [12]:
# Clean Year



# dump recweek data
recweek

,firstname,lastname,birthmonth,birthday,birthyear,course,yearlevel,phonenumber
0,name1,lastname1,November,26,1998,BS LM,2,+63 918 302 6210
1,name2,lastname2,May,26,2001,Management,4,927 212 6276
2,name3,lastname3,May,16,1997,BS CTM,1,+63 918 253 7017
3,name4,lastname4,December,9,1997,BS ITE,3,918 714 4360
4,name5,lastname5,September,5,1997,BS CTM,3,+63 927 462 1674
...,...,...,...,...,...,...,...,...
2995,name2996,lastname2996,March,28,1997,BS CTM,2,917 795 5051
2996,name2997,lastname2997,May,19,1998,LM,2,917 445 3378
2997,name2998,lastname2998,July,3,1998,ITE,2,917 154 5694
2998,name2999,lastname2999,December,2,1998,ME,3,927 497 8977


In [14]:
# Clean Course





# dump recweek data
recweek

,firstname,lastname,birthmonth,birthday,birthyear,course,yearlevel,phonenumber
0,name1,lastname1,November,26,1998,BS Legal Management,2,+63 918 302 6210
1,name2,lastname2,May,26,2001,BS Management,4,927 212 6276
2,name3,lastname3,May,16,1997,BS Communications Technology,1,+63 918 253 7017
3,name4,lastname4,December,9,1997,BS IT Entrepreneurship,3,918 714 4360
4,name5,lastname5,September,5,1997,BS Communications Technology,3,+63 927 462 1674
...,...,...,...,...,...,...,...,...
2995,name2996,lastname2996,March,28,1997,BS Communications Technology,2,917 795 5051
2996,name2997,lastname2997,May,19,1998,BS Legal Management,2,917 445 3378
2997,name2998,lastname2998,July,3,1998,BS IT Entrepreneurship,2,917 154 5694
2998,name2999,lastname2999,December,2,1998,BS Management Engineering,3,927 497 8977


In [15]:
# mobile number cleaning

# dump recweek data
recweek

,firstname,lastname,birthmonth,birthday,birthyear,course,yearlevel,phonenumber
0,name1,lastname1,November,26,1998,BS Legal Management,2,0918 302 6210
1,name2,lastname2,May,26,2001,BS Management,4,0927 212 6276
2,name3,lastname3,May,16,1997,BS Communications Technology,1,0918 253 7017
3,name4,lastname4,December,9,1997,BS IT Entrepreneurship,3,0918 714 4360
4,name5,lastname5,September,5,1997,BS Communications Technology,3,0927 462 1674
...,...,...,...,...,...,...,...,...
2995,name2996,lastname2996,March,28,1997,BS Communications Technology,2,0917 795 5051
2996,name2997,lastname2997,May,19,1998,BS Legal Management,2,0917 445 3378
2997,name2998,lastname2998,July,3,1998,BS IT Entrepreneurship,2,0917 154 5694
2998,name2999,lastname2999,December,2,1998,BS Management Engineering,3,0927 497 8977


In [5]:
# write code here

# Converting birthyear column into object/string data for regex applications
recweek["birthyear"] = recweek["birthyear"].astype(str)

# Function used for cleaning "years" via vectorization. Only 2 cases:
# Case 1 - Year is written as two digits, which signifies late 20th century births (97)
# Case 2 - Year is written in full (1997,2001,etc.)

def cleanyear(year):
    twodigit = r"^\d{2}$"
    
    if (re.match(twodigit,year)):
        year = "19" + year
        return year
    else:
        return year

# Function used for cleaning "courses" via vectorization. Only 4 cases:
# Case 1 - Whole name of the course is stated (Management)
# Case 2 - Abbreviated course (MGT)
# Case 3 - Abreviated course with 'BS' (BS MGT)
# Case 4 - Full course name - (BS Management)

def cleancourse(course):
    coursedict = {
        "ME":"BS Management Engineering","BS ME":"BS Management Engineering","Management Engineering":"BS Management Engineering","BS Management Engineering":"BS Management Engineering",
        "CTM":"BS Communications Technology","BS CTM":"BS Communications Technology","Communications Technology":"BS Communications Technology","BS Communications Technology":"BS Communications Technology",
        "MGT":"BS Management","BS MGT":"BS Management","Management":"BS Management","BS Management":"BS Management",
        "LM":"BS Legal Management","BS LM":"BS Legal Management","Legal Management":"BS Legal Management","BS Legal Management":"BS Legal Management",
        "ITE":"BS IT Entrepreneurship","BS ITE":"BS IT Entrepreneurship","Information Technology Entrepreneurship":"BS IT Entrepreneurship","BS IT Entrepreneurship":"BS IT Entrepreneurship"}
    
    for c in coursedict:
        if course==c:
            return str(coursedict[c])
        else:
            continue

# Function used for cleaning "phonenumbers" via vectorization. Only 2 cases:
# Case 1 - Unique digits (917 654 3210)
# Case 2 - Country code included (+63 917 654 3210)

def cleanphonenumber(phonenumber):
    
    uniquepattern = r"^9[\d]{2} \d{3} \d{4}$"
    codedpattern = r"^\+63 9[\d]{2} \d{3} \d{4}"
    
    if (re.match(uniquepattern,phonenumber)):
        phonenumber = "0"+phonenumber
        return phonenumber
    elif (re.match(codedpattern,phonenumber)):
        phonenumber = "0" + phonenumber[4:]
        return phonenumber
    else:
        return phonenumber


# Cleaning columns via vectorization:
recweek["birthyear"] = recweek["birthyear"].apply(cleanyear)             # cleaning birth years
recweek["course"] = recweek["course"].apply(cleancourse)                 # cleaning courses
recweek["phonenumber"] = recweek["phonenumber"].apply(cleanphonenumber)  # cleaning phone numbers

# Dumping dataframe
recweek


,firstname,lastname,birthmonth,birthday,birthyear,course,yearlevel,phonenumber
0,name1,lastname1,November,26,1998,BS Legal Management,2,0918 302 6210
1,name2,lastname2,May,26,2001,BS Management,4,0927 212 6276
2,name3,lastname3,May,16,1997,BS Communications Technology,1,0918 253 7017
3,name4,lastname4,December,9,1997,BS IT Entrepreneurship,3,0918 714 4360
4,name5,lastname5,September,5,1997,BS Communications Technology,3,0927 462 1674
...,...,...,...,...,...,...,...,...
2995,name2996,lastname2996,March,28,1997,BS Communications Technology,2,0917 795 5051
2996,name2997,lastname2997,May,19,1998,BS Legal Management,2,0917 445 3378
2997,name2998,lastname2998,July,3,1998,BS IT Entrepreneurship,2,0917 154 5694
2998,name2999,lastname2999,December,2,1998,BS Management Engineering,3,0927 497 8977


In [129]:
# add genders
# hint: load "studentprofiles.csv" as new dataframe and add to our dataframe using Pandas merge(...) function


# dump recweek data
final_recweek

,firstname,lastname,gender,birthmonth,birthday,birthyear,course,yearlevel,phonenumber
0,name1,lastname1,Female,November,26,1998,BS Legal Management,2,0918 302 6210
1,name2,lastname2,Female,May,26,2001,BS Management,4,0927 212 6276
2,name3,lastname3,Female,May,16,1997,BS Communications Technology,1,0918 253 7017
3,name4,lastname4,Male,December,9,1997,BS Information Technology Entrepreneurship,3,0918 714 4360
4,name5,lastname5,Female,September,5,1997,BS Communications Technology,3,0927 462 1674
...,...,...,...,...,...,...,...,...,...
2995,name2996,lastname2996,Female,March,28,1997,BS Communications Technology,2,0917 795 5051
2996,name2997,lastname2997,Male,May,19,1998,BS Legal Management,2,0917 445 3378
2997,name2998,lastname2998,Female,July,3,1998,BS Information Technology Entrepreneurship,2,0917 154 5694
2998,name2999,lastname2999,Male,December,2,1998,BS Management Engineering,3,0927 497 8977


In [6]:
# write code here

# Loading studentprofiles data as dataframe
filename2 = "studentprofiles.csv"
studentprofiles = pd.read_csv(filename2)

# Isolating the columns in "studentprofiles" dataframe to match based on "firstname" with "recweek" and merge dataframes
genderdf = studentprofiles[["firstname","gender"]]

# Merging dataframes
final_recweek = recweek.merge(genderdf, on="firstname")

# Rearranging the columns in the new merged dataframe
final_recweek = final_recweek[["firstname","lastname","gender","birthmonth","birthday","birthyear","course","yearlevel","phonenumber"]]

# Dump
final_recweek


,firstname,lastname,gender,birthmonth,birthday,birthyear,course,yearlevel,phonenumber
0,name1,lastname1,Female,November,26,1998,BS Legal Management,2,0918 302 6210
1,name2,lastname2,Female,May,26,2001,BS Management,4,0927 212 6276
2,name3,lastname3,Female,May,16,1997,BS Communications Technology,1,0918 253 7017
3,name4,lastname4,Male,December,9,1997,BS IT Entrepreneurship,3,0918 714 4360
4,name5,lastname5,Female,September,5,1997,BS Communications Technology,3,0927 462 1674
...,...,...,...,...,...,...,...,...,...
2995,name2996,lastname2996,Female,March,28,1997,BS Communications Technology,2,0917 795 5051
2996,name2997,lastname2997,Male,May,19,1998,BS Legal Management,2,0917 445 3378
2997,name2998,lastname2998,Female,July,3,1998,BS IT Entrepreneurship,2,0917 154 5694
2998,name2999,lastname2999,Male,December,2,1998,BS Management Engineering,3,0927 497 8977


In [7]:
# export to csv
# compare with cleanrecweekdata

final_recweek.to_csv("cleanrecweekdata.csv")
!cat cleanrecweekdata.csv


,firstname,lastname,gender,birthmonth,birthday,birthyear,course,yearlevel,phonenumber
0,name1,lastname1,Female,November,26,1998,BS Legal Management,2,0918 302 6210
1,name2,lastname2,Female,May,26,2001,BS Management,4,0927 212 6276
2,name3,lastname3,Female,May,16,1997,BS Communications Technology,1,0918 253 7017
3,name4,lastname4,Male,December,9,1997,BS IT Entrepreneurship,3,0918 714 4360
4,name5,lastname5,Female,September,5,1997,BS Communications Technology,3,0927 462 1674
5,name6,lastname6,Female,March,2,1999,BS Management,2,0918 629 5268
6,name7,lastname7,Female,November,10,1998,BS Legal Management,1,0917 369 7388
7,name8,lastname8,Male,August,28,1999,BS Management Engineering,3,0917 240 6000
8,name9,lastname9,Male,July,10,1997,BS Management Engineering,3,0918 226 5748
9,name10,lastname10,Female,September,14,2001,BS Communications Technology,1,0918 342 1717
10,name11,lastname11,Male,December,8,1998,BS Legal Management,4,0927 708 1308
11,name12,lastname12,Male,May,4,1997,BS Management

In [4]:
# Use ls (Mac) or dir (Windows) cleanrecweekdata.csv
# choose appropriate command
# !dir cleanrecweekdata.csv
# or
# !ls cleanrecweekdata.csv

 Volume in drive D is UserProfile
 Volume Serial Number is 508F-1BD0

 Directory of D:\Users\jilagan\Dropbox\Workspaces\src\ITMGT25.03_Intersession2020\tests\EM3

08/10/2020  04:01 PM           276,118 cleanrecweekdata.csv
               1 File(s)        276,118 bytes
               0 Dir(s)  44,855,271,424 bytes free


In [8]:
# write code here
!ls cleanrecweekdata.csv


cleanrecweekdata.csv


#### Get statistics

In [51]:
# People per Birthmonth sorted by calendar numeric month


,birthmonth,count
0,January,270
1,February,232
2,March,246
3,April,267
4,May,273
5,June,259
6,July,249
7,August,241
8,September,232
9,October,178


In [9]:
# Write code here

# Creating a list that orders the months, since the "value_counts" has no way of automating the order of
# months displayed. Recalling each of the months in a loop can then create the desired sort:
months = ["January","February","March","April","May","June","July","August","September","October","November","December"]

# Getting the counts of each month in all 3000 rows via '.value_counts'
monthcounts = final_recweek["birthmonth"].value_counts(sort=False)

# List for lists made of pairs for the data frame creation
monthcountpairs = []

# Assigning each month to its corresponding number count pair
for m in months:
    pair = [m,monthcounts[m]]
    monthcountpairs.append(pair)

# Creating the birthmonth counts data frame 
birthmonthcount = pd.DataFrame(monthcountpairs,columns=["birthmonth","count"])

# Dumping the data frame
birthmonthcount


,birthmonth,count
0,January,270
1,February,232
2,March,246
3,April,267
4,May,273
5,June,259
6,July,249
7,August,241
8,September,232
9,October,178


In [52]:
# same day
# sort by calendar day


,birthday,count
0,1,112
1,2,106
2,3,90
3,4,110
4,5,100
5,6,99
6,7,88
7,8,111
8,9,98
9,10,111


In [10]:
# write code here

# First we get the list of all the numeric days present in the dataframe, then sort days in ascending order:
uniquedays = final_recweek["birthday"].unique()
uniquedays.sort()

# Same process as before...
daycounts = final_recweek["birthday"].value_counts(sort=True)

daycountpairs = []

for d in uniquedays:
    pair = [d,daycounts[d]]
    daycountpairs.append(pair)

# Creating the birthday counts dataframe
birthdaycount = pd.DataFrame(daycountpairs,columns=["birthday","count"])

# Dump
birthdaycount

,birthday,count
0,1,112
1,2,106
2,3,90
3,4,110
4,5,100
5,6,99
6,7,88
7,8,111
8,9,98
9,10,111


In [53]:
# same year
# sort accordingly


,birthyear,count
0,1997,756
1,1998,746
2,1999,756
3,2000,380
4,2001,362


In [11]:
# write code here

uniqueyears = final_recweek["birthyear"].unique()
uniqueyears.sort()

yearcounts = final_recweek["birthyear"].value_counts(sort=True)

yearcountpairs = []

for y in uniqueyears:
    pair = [y,yearcounts[y]]
    yearcountpairs.append(pair)
    
# Creating the birthyear counts dataframe
birthyearcount = pd.DataFrame(yearcountpairs,columns=["birthyear","count"])

# Dump
birthyearcount


,birthyear,count
0,1997,756
1,1998,746
2,1999,756
3,2000,380
4,2001,362


In [55]:
# same year level
# sort accordingly


,yearlevel,count
0,1,760
1,2,791
2,3,751
3,4,698


In [12]:
# write code here

yearlevels = final_recweek["yearlevel"].unique()
yearlevels.sort()

levelcounts = final_recweek["yearlevel"].value_counts(sort=True)

yearlevelpairs = []

for y in yearlevels:
    pair = [y,levelcounts[y]]
    yearlevelpairs.append(pair)
    
# Creating the yearlevel counts dataframe
yearlevelcounts = pd.DataFrame(yearlevelpairs,columns=["yearlevel","count"])

# Dump
yearlevelcounts


,yearlevel,count
0,1,760
1,2,791
2,3,751
3,4,698


In [56]:
# same course
# sorting does not matter


,course,count
0,BS Communications Technology,589
1,BS IT Entrepreneurship,590
2,BS Legal Management,585
3,BS Management,629
4,BS Management Engineering,607


In [13]:
# write code here

courses = final_recweek["course"].unique()

coursecounts = final_recweek["course"].value_counts(sort=True)

coursecountpairs = []

for c in courses:
    pair = [c,coursecounts[c]]
    coursecountpairs.append(pair)

# Creating the course counts dataframe
coursescountdataframe = pd.DataFrame(coursecountpairs,columns=["course","count"])

# Dump
coursescountdataframe



,course,count
0,BS Legal Management,585
1,BS Management,629
2,BS Communications Technology,589
3,BS IT Entrepreneurship,590
4,BS Management Engineering,607


In [59]:
# choose any of the following two outputs:

carrier  gender
Globe    Female     984
         Male      1061
Smart    Female     494
         Male       461
Name: firstname, dtype: int64

In [61]:
# or

,carrier,gender,count
0,Globe,Female,984
1,Globe,Male,1061
2,Smart,Female,494
3,Smart,Male,461


In [62]:
# carrier count

,carrier,count
0,Globe,2045
1,Smart,955


In [14]:
# write code here

# Globe (0917 or 0927)
# SMART (0918)

# This code is made in such a way that newer inputs of different carriers and their numbers can be accounted for

genders = ["Male","Female"]

# Regex patterns for vectorization function
globepattern = r"^09[1-2]7"
smartpattern = r"0918"

# Vectorization function that determines if a 'phonenumber' is a SMART, Globe, or any other added carrier
def smartorglobe(number):
    if (re.match(globepattern,number)):
        return "Globe"
    elif (re.match(smartpattern,number)):
        return "Smart"
    else:
        return "No carrier match"

# Creating a new dataframe from the gender and phonenumber columns in final_recweek
cellcarrierdf = final_recweek[["gender","phonenumber"]].copy()

# Vectorizing into a new data frame column 'carrier'
cellcarrierdf["carrier"] = cellcarrierdf["phonenumber"].apply(smartorglobe)

# Makes a list of all the present carriers in the dataframe 
carriers = cellcarrierdf["carrier"].unique()



In [15]:
# Creating the gender specific dataframe:

gendercarrierrows = []

for c in carriers:
    for g in genders:
        referencedataframe = cellcarrierdf[cellcarrierdf["carrier"].str.contains(c) & cellcarrierdf["gender"].str.contains(g)]
        row = [c,g,len(referencedataframe)]
        gendercarrierrows.append(row)

gendercarrierdf = pd.DataFrame(gendercarrierrows,columns=["carrier","gender","count"])

# Dump
gendercarrierdf

,carrier,gender,count
0,Smart,Male,461
1,Smart,Female,494
2,Globe,Male,1061
3,Globe,Female,984


In [16]:
# Creating the carrier count dataframe:

carrierrows = []

for c in carriers:
    count = cellcarrierdf[cellcarrierdf["carrier"].str.contains(c)]
    row = [c,len(count)]
    carrierrows.append(row)

carriercount = pd.DataFrame(carrierrows,columns=["carrier","count"])

# Dump
carriercount

,carrier,count
0,Smart,955
1,Globe,2045
